In [1]:
"""
Units of quantities--->
torque: N/m
pressures: bar
air/water flow: Lt/min
noise: db
ground accelerations along x,y,z axes (gaxxy,gaccy,gaccz): m/s^2
head accelerations (haxxy,haccy,haccz) along x,y,z axes: m/s^2
rpm: revolutions/min
powers: Watt
temperatures: (degree)C
source: https://www.kaggle.com/datasets/afumetto/predictive-maintenance-dataset-air-compressor?select=data.csv
"""

'\nUnits of quantities--->\ntorque: N/m\npressures: bar\nair/water flow: Lt/min\nnoise: db\nground accelerations along x,y,z axes (gaxxy,gaccy,gaccz): m/s^2\nhead accelerations (haxxy,haccy,haccz) along x,y,z axes: m/s^2\nrpm: revolutions/min\npowers: Watt\ntemperatures: (degree)C\nsource: https://www.kaggle.com/datasets/afumetto/predictive-maintenance-dataset-air-compressor?select=data.csv\n'

In [2]:
# Load dataset
import pandas as pd
df=pd.read_csv("processed_data.csv")

In [3]:
df.head()

,id,rpm,motor_power,torque,outlet_pressure_bar,air_flow,noise_db,outlet_temp,wpump_outlet_press,water_inlet_temp,...,gaccy,gaccz,haccx,haccy,haccz,bearings,wpump,radiator,exvalve,acmotor
0,1,499,1405.842858,27.511708,1.000,616.579758,40.840517,78.554715,2.960632,43.166392,...,0.383773,2.649801,1.213344,1.409218,2.962484,0,0,0,0,0
1,2,513,1457.370092,31.030115,1.081,615.667472,40.484226,76.902822,2.536711,47.342143,...,0.450954,2.669423,1.210674,1.379050,2.938135,0,0,0,0,0
2,3,495,1582.249959,33.484653,1.369,614.755186,40.918572,77.547021,2.112789,49.306593,...,0.443924,2.772009,1.210612,1.373490,2.991878,0,0,0,0,0
3,4,480,1712.466820,36.394475,1.691,613.950497,40.450953,80.059949,2.087534,46.886933,...,0.370457,2.876056,1.213223,1.443234,3.096158,0,0,0,0,0
4,5,498,1766.035170,38.249154,1.731,613.664264,41.233739,79.130424,2.338877,50.498100,...,0.383868,2.849451,1.209216,1.405190,3.059417,0,0,0,0,0


In [4]:
# Dropping unimportant features
df.drop(['id','oilpump_power','gaccx','gaccy','gaccz','haccx','haccy','haccz','acmotor'],axis=1,inplace=True)

In [5]:
from sklearn.model_selection import train_test_split
X = df.drop(['outlet_temp'],axis=1)
y = df['outlet_temp']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scaling
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [6]:
# Reshaping train data for training
X_train=X_train.to_numpy()
X_train=X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test=X_test.to_numpy()
X_test=X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Define the model
model = Sequential()
model.add(LSTM(2000, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))  # Input Layer
model.add(LSTM(2000, return_sequences=True))  # Hidden Layer 1
model.add(LSTM(2000, return_sequences=True))  # Hidden Layer 2
model.add(LSTM(2000, return_sequences=True))  # Hidden Layer 3
model.add(LSTM(2000, return_sequences=False))  # Hidden Layer 4
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='rmsprop', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=100)  # Adjust batch_size as needed

Epoch 1/50
8/8 [==============================] - 59s 6s/step - loss: 7800.0132
Epoch 2/50
8/8 [==============================] - 50s 6s/step - loss: 2841.1277
Epoch 3/50
8/8 [==============================] - 40s 5s/step - loss: 1788.8320
Epoch 4/50
8/8 [==============================] - 42s 5s/step - loss: 1135.3173
Epoch 5/50
8/8 [==============================] - 41s 5s/step - loss: 735.6398
Epoch 6/50
8/8 [==============================] - 40s 5s/step - loss: 507.4107
Epoch 7/50
8/8 [==============================] - 41s 5s/step - loss: 403.8733
Epoch 8/50
8/8 [==============================] - 40s 5s/step - loss: 372.2896
Epoch 9/50
8/8 [==============================] - 41s 5s/step - loss: 364.6523
Epoch 10/50
8/8 [==============================] - 54s 7s/step - loss: 364.5385
Epoch 11/50
8/8 [==============================] - 44s 5s/step - loss: 364.6145
Epoch 12/50
8/8 [==============================] - 42s 5s/step - loss: 366.8611
Epoch 13/50
8/8 [============================

In [8]:
# Predicting on test data
predictions = model.predict(X_test)

8/8 [==============================] - 6s 332ms/step


In [9]:
# Calculating RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
rmse=np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 19.32760585241494
